In [ ]:
versioninfo()

In [ ]:
g=6.0
ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

In [ ]:
ENV["JULIA_DEBUG"]="all"

# New version of main

- [X] refactor the code of main1
- [X] use seeding
- [X] pre-gen graphs and splitting
- [X] use cloud GPU for training
- [X] I don't need correlation experiments anymore

In [ ]:
include("main.jl")

In [ ]:
main_ch3()

In [ ]:
main()

Ensemble

- [ ] I actually probably want to merge the datasets with the same d.
  This is because, I can merge the data to be more evenly distributed to fit the model.
- [ ] I probably want to adjust batch size to speed it up

# Debug weird problem at Nayuki

In [ ]:
spec = DataSpec(d=50, k=1, gtype=:SF,
                    noise=:Gaussian, mat=:COV)

In [ ]:
d = 50
k = 1
specs = []
for gtype in [:ER, :SF],
    k in [1]
    push!(specs, DataSpec(d=d, k=k, gtype=gtype,
            noise=:Gaussian, mat=:COV))
end
specs = Array{DataSpec}(specs)

In [ ]:
ds, test_ds = spec2ds(spec)

In [ ]:
specs[1].bsize

In [ ]:
merge_dses

In [ ]:
ds, test_ds = spec2ds(specs, merge=true)

In [ ]:
x, y = next_batch!(test_ds) |> gpu;

In [ ]:
size(x)

In [ ]:
model = deep_eq_model_fn() |> gpu;

In [ ]:
model(x)

In [ ]:
gradient(()->sum(model(x)), Flux.params(model))

# Debug eq2_model

In [ ]:
# 1. load the data
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:CH3)
ds, test_ds = spec2ds(spec)
x, y = next_batch!(test_ds) |> gpu

In [ ]:

# 2. load the model
model = eq2_model_fn() |> gpu

In [ ]:
# 3. apply the model on data
model(x)
# 4. calculate the gradient

In [ ]:
cpu(model)(cpu(x))

In [ ]:
size(x)

In [ ]:
cpux = cpu(x)

In [ ]:
model[1](cpux)

# Sachs 2005 experiment

## Load and explore the data

In [ ]:
import CSV

In [ ]:
df = CSV.read("Sachs/1.cd3cd28.csv")

In [ ]:
SachsX = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
SachsG = Sachs_ground_truth()

In [ ]:
myplot(SachsG)

In [ ]:
medcovX = cov(SachsX) ./ median(var(SachsX, dims=1))

In [ ]:
maxcovX = cov(SachsX) ./ maximum(var(SachsX, dims=1))

In [ ]:
corX = cor(SachsX)

In [ ]:
ch2X = getch2(SachsX)

In [ ]:
include("exp.jl")

## Load a model

In [ ]:
# load the trained model
@load "saved_models/EQ-d=20_k=1_gtype=SF_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/EQ-d=10_k=1_gtype=ER_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "back/back-0907/CORCOV/EQ-d=10_k=1_gtype=SF_noise=Gaussian_mat=COR_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/ensK-2020-09-08T10:58:41.247-ensemble/step-10000.bson" model

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
@load "saved_models/ensemEQ-CH2-1,2,4-2020-10-11T11:29:01.183-ensemble/step-100000.bson" model

## Evaluate the model

In [ ]:
out = inf_one(model, medcovX)

In [ ]:
out = inf_one(model, corX)

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
out = inf_one(model, ch2X)

In [ ]:
Wout = threshold(σ.(out), 0.3, true)

In [ ]:
myplot(DiGraph(Wout), names(df))

In [ ]:
# FIXME NOW !!!! the names might not match at all!!!
myplot(SachsG, names(df))

In [ ]:
# predicted edge, true edge, SHD
predicted_edge = ne(DiGraph(Wout))
@show predicted_edge
correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])
@show correct_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(Wout, ytrue)

In [ ]:
# TODO calculate #reverse direction edges
sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])

In [ ]:
sum(Wout'[Wout' .== 1] .== adjacency_matrix(SachsG)[Wout' .== 1])

In [ ]:
# TODO implement the recursive add procedure to remove cycles
# Or, just construct the graph, and keep removing until it is a DAG
is_cyclic(DiGraph(Wout))

In [ ]:
Wout

In [ ]:
# Or just implement the procedure
out

## Adding non-cyclic procedure

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
# add sorted idx
g = MetaDiGraph(11)
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
p1 = myplot(g, names(df))

In [ ]:
draw(PNG("p1.png"), p1)

In [ ]:
p0 = myplot(SachsG, names(df))

In [ ]:
draw(PNG("p0.png"), p0)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(SachsG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(adj1, ytrue)

## More data

- [ ] goblinop's gaussian data (just for test)
- [ ] bnlearn's continous data
- [ ] discrete data?

### goblinop's (??) gaussian data

In [ ]:
import CSV

In [ ]:
df = CSV.read("data-back/gaussian.dat", delim=" ")

In [ ]:
X = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
maxcovX = cov(X) ./ maximum(var(X, dims=1))

In [ ]:
include("exp.jl")

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
size(X,2)

In [ ]:
# add sorted idx
g = MetaDiGraph(size(X,2))
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
bnp1 = myplot(g, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
function bnlearn_ground_truth()
    greal = named_graph([:A, :B, :C, :D, :E, :F, :G])
    named_graph_add_edge!(greal, :B, :C)
    named_graph_add_edge!(greal, :A, :C)
    named_graph_add_edge!(greal, :B, :D)
    named_graph_add_edge!(greal, :D, :F)
    named_graph_add_edge!(greal, :A, :F)
    named_graph_add_edge!(greal, :G, :F)
    named_graph_add_edge!(greal, :E, :F)
    greal
end

In [ ]:
groundG = bnlearn_ground_truth()

In [ ]:
bnp0 = myplot(groundG, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
draw(PNG("bnp0.png"), bnp0)

In [ ]:
draw(PNG("bnp1.png"), bnp1)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(groundG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(groundG))
sup_graph_metrics(adj1, ytrue)